# Building a Voice-Enabled AI Assistant with Gradio

This code creates an interactive AI assistant that can both speak and listen. Users can type messages or record their voice, and the AI responds with natural-sounding speech.


```text
┌─────────────────────────────────────────────────────────┐
│                    USER INPUT                           │
│  ┌──────────────┐              ┌──────────────┐         │
│  │ Type Text    │      OR      │ Speak/Upload │         │
│  └──────┬───────┘              └──────┬───────┘         │
│         │                             │                 │
│         │                             ▼                 │
│         │                    ┌────────────────┐         │
│         │                    │ Whisper STT    │         │
│         │                    │ (voice→text)   │         │
│         │                    └────────┬───────┘         │
│         └──────────────┬──────────────┘                 │
│                        ▼                                │
└────────────────────────┼────────────────────────────────┘
                         │
            ┌────────────▼───────────┐
            │   Unified Text Input   │
            └────────────┬───────────┘
                         │
┌────────────────────────▼────────────────────────────────┐
│              CONVERSATION MEMORY                        │
│  ┌──────────────────────────────────────────┐           │
│  │ History = [                              │           │
│  │   {role: "user", content: "Hi"},         │           │
│  │   {role: "assistant", content: "Hello!"}, │          │
│  │   ... previous messages ...              │           │
│  │ ]                                        │           │
│  └──────────────────┬───────────────────────┘           │
│                     │                                   │
│                     ▼                                   │
│  ┌──────────────────────────────────────────┐           │
│  │ Add new user message to history          │           │
│  └──────────────────┬───────────────────────┘           │
└─────────────────────┼───────────────────────────────────┘
                      │
         ┌────────────▼──────────────┐
         │   Send to GPT-4o-mini     │
         │   (entire conversation)   │
         └────────────┬──────────────┘
                      │
         ┌────────────▼──────────────┐
         │   AI generates response   │
         └────────────┬──────────────┘
                      │
         ┌────────────▼──────────────┐
         │   Add AI response to      │
         │   conversation history    │
         └────────────┬──────────────┘
                      │
┌─────────────────────▼─────────────────────────────────┐
│                   OUTPUT                              │
│         ┌─────────────────────────┐                   │
│         │  Display text response  │                   │
│         └───────────┬─────────────┘                   │
│                     │                                 │
│         ┌───────────▼─────────────┐                   │
│         │  Convert to speech      │                   │
│         │  (OpenAI TTS API)       │                   │
│         └───────────┬─────────────┘                   │
│                     │                                 │
│         ┌───────────▼─────────────┐                   │
│         │  Play audio (pygame)    │                   │
│         └─────────────────────────┘                   │
└───────────────────────────────────────────────────────┘
```


In [ ]:
import os
import gradio as gr
from openai import OpenAI
from datetime import datetime
import pygame

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pygame.mixer.init()

def speak(text, voice="alloy"):
    try:
        speech_file = os.path.join('saved audio', f"speech_{datetime.now().timestamp()}.mp3")
        
        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice=voice,
            input=text
        ) as response:
            response.stream_to_file(speech_file)
        
        pygame.mixer.music.load(speech_file)
        pygame.mixer.music.play()
        
        try:
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)
        except KeyboardInterrupt:
            pygame.mixer.music.stop()
        
        os.remove(speech_file)
    except Exception as e:
        print(f"TTS Error: {e}")

# Whisper Transcription
def transcribe_audio(audio_file):
    try:
        with open(audio_file, "rb") as f:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=f
            )
        return transcript.text
    except Exception as e:
        print(f"Transcription error: {e}")
        return ""

# Chat Interface
def chat_interface(text_input, chat_history, voice_choice, audio_file=None):
    
    # Get user input from text or audio
    user_input = text_input
    if audio_file:
        user_input = transcribe_audio(audio_file)
        print(f"Transcribed audio: {user_input}")

    if not user_input or not user_input.strip():
        return chat_history, ""

    # Initialize chat history if None
    if chat_history is None:
        chat_history = []

    messages = [{"role": "system", "content": "You are a helpful, natural AI assistant."}]
    
    # Handle both old list format [[user, ai]] and new dict format
    for msg in chat_history:
        if isinstance(msg, dict):
            # New Gradio format
            messages.append(msg)
        elif isinstance(msg, (list, tuple)) and len(msg) == 2:
            # Old Gradio format - convert to OpenAI format
            messages.append({"role": "user", "content": msg[0]})
            messages.append({"role": "assistant", "content": msg[1]})
    
    messages.append({"role": "user", "content": user_input})

    # Get AI response
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        ai_reply = response.choices[0].message.content
    except Exception as e:
        ai_reply = f"Error: {e}"

    # Speak the response
    speak(ai_reply, voice_choice)

    chat_history.append({"role": "user", "content": user_input})
    chat_history.append({"role": "assistant", "content": ai_reply})

    return chat_history, ""


# Gradio App
with gr.Blocks() as demo:
    gr.Markdown("# 🎙️ Human-Like AI Voice Assistant")
    gr.Markdown("Type or speak your message, and the AI will respond with voice!")
    
    chatbot = gr.Chatbot()
    
    with gr.Row():
        with gr.Column(scale=4):
            txt = gr.Textbox(
                placeholder="Type your message here...", 
                label="Text Input",
                lines=2
            )
        with gr.Column(scale=1):
            voice = gr.Dropdown(
                ["alloy", "echo", "fable", "onyx", "nova", "shimmer"], 
                value="alloy", 
                label="Voice"
            )
    
    audio_input = gr.Audio(label="Or record/upload audio", type="filepath")
    
    send_btn = gr.Button("Send", variant="primary")
    
    send_btn.click(
        chat_interface, 
        inputs=[txt, chatbot, voice, audio_input], 
        outputs=[chatbot, txt]
    )
    
    txt.submit(
        chat_interface, 
        inputs=[txt, chatbot, voice, audio_input], 
        outputs=[chatbot, txt]
    )

demo.launch()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.11)
Hello from the pygame community. https://www.pygame.org/contribute.html
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Transcribed audio: Hi, how are you? How's it going?
Transcribed audio: I want to purchase a product, so could you please help me with that?


<span style="color:red;">> **IMPORTANT NOTE**: Run the local url to use the audio feature</span>

This guide breaks down how each part works and why it's structured this way.

# Part One: Setting Up Your Tools

Before building the interface, you need to import the tools your application will use and initialize the services that power your assistant.


```python
import os
import gradio as gr
from openai import OpenAI
from datetime import datetime
import pygame

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pygame.mixer.init()
```


# Part Two: Teaching Your Assistant to Speak

The first major function converts text into spoken audio using OpenAI's text-to-speech (TTS) service.


```python
def speak(text, voice="alloy"):
    try:
        speech_file = os.path.join('saved audio', f"speech_{datetime.now().timestamp()}.mp3")

        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice=voice,
            input=text
        ) as response:
            response.stream_to_file(speech_file)
```


### Breaking it down

- The function takes two parameters: `text` (what to say) and `voice` (which voice to use), with `"alloy"` set as the default voice.
- It creates a unique filename using the current timestamp and saves the audio file inside a dedicated _saved audio_ folder.
- The `with` statement requests OpenAI to generate speech and uses streaming to efficiently handle the incoming audio data.
- The `response.stream_to_file()` method writes the generated speech directly to an MP3 file on disk.

The `client` object represents your connection to OpenAI’s API. It automatically reads the API key from an environment variable, which is a secure and recommended way to handle sensitive credentials without hard-coding them into your source code. The `pygame.mixer.init()` line initializes the audio subsystem, making sure your application is ready to load and play sound files such as generated speech.

### Part Two: Teaching Your Assistant to Speak

The first major function focuses on converting text into spoken audio using OpenAI’s text-to-speech (TTS) service. This function sends a text prompt to the TTS model, receives synthesized speech as audio data, and saves it as a playable sound file. Once generated, the audio can be played back immediately, allowing the assistant to respond in a natural, human-like voice rather than plain text.


```python
def speak(text, voice="alloy"):
    try:
        speech_file = os.path.join('saved audio', f"speech_{datetime.now().timestamp()}.mp3")

        with client.audio.speech.with_streaming_response.create(
            model="tts-1",
            voice=voice,
            input=text
        ) as response:
            response.stream_to_file(speech_file)
```


### Breaking It Down

The function accepts two parameters: `text`, which represents the content that will be spoken, and `voice`, which specifies the voice style to use. If no voice is provided, it defaults to `"alloy"`, ensuring consistent behavior without requiring extra input from the user.

To avoid filename conflicts, the function generates a unique filename using the current timestamp and saves the output inside a dedicated “saved audio” folder. This makes each generated audio file distinct and easy to manage or replay later if needed.

The `with` statement is used to request speech generation from OpenAI while enabling streaming. Streaming allows the audio data to be processed efficiently as it is generated, rather than waiting for the entire file to be created at once. Finally, `response.stream_to_file()` writes the streamed audio data directly to an MP3 file, producing a ready-to-play speech output.


```python
        pygame.mixer.music.load(speech_file)
        pygame.mixer.music.play()

        try:
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)
        except KeyboardInterrupt:
            pygame.mixer.music.stop()

        os.remove(speech_file)
```


### What Happens Next

Once the audio file is generated, `load()` prepares it for playback by loading it into the audio system’s memory. Calling `play()` then starts the audio output, allowing the assistant’s response to be heard through the speakers. A `while` loop continuously checks whether the audio is still playing, pausing briefly (every 10 milliseconds) to keep the program responsive without consuming unnecessary CPU resources.

If you press **Ctrl+C** during playback, the program immediately stops, giving you manual control to interrupt the audio at any time. After the playback finishes (or is interrupted), `os.remove()` deletes the temporary audio file, ensuring disk space is not wasted by leftover files.

The entire function is wrapped in a `try/except` block, which allows any unexpected errors—such as missing audio devices or file issues—to be handled gracefully. Instead of crashing the application, the function prints a clear error message and continues running.

### Part Three: Teaching Your Assistant to Listen

This next function focuses on the opposite direction of communication: turning spoken input into text. It records audio from the microphone and sends it to OpenAI’s **Whisper** speech recognition model, which transcribes the audio into readable text. This transcription is then passed to the assistant as user input, enabling natural, voice-driven conversations.


```python
def transcribe_audio(audio_file):
    try:
        with open(audio_file, "rb") as f:
            transcript = client.audio.transcriptions.create(
                model="whisper-1",
                file=f
            )
        return transcript.text
    except Exception as e:
        print(f"Transcription error: {e}")
        return ""
```


### How It Works

The transcription function starts by taking the path to an audio file as its input. Because audio files are binary data rather than plain text, the file is opened in **read-binary (`"rb"`) mode**, which ensures the raw audio bytes are handled correctly. This audio data is then sent to OpenAI’s **Whisper** speech recognition model, which analyzes the sound and converts spoken words into text. Once the transcription is complete, the function returns the recognized text so it can be used as normal user input.

To keep the application stable, the function is wrapped in error handling logic. If anything goes wrong—such as a missing file, microphone issue, or API error—it safely returns an empty string instead of crashing the entire program. Compared to the text-to-speech function, this process is simpler because it only sends data to OpenAI and waits for text in return, without needing to manage audio playback or file cleanup.

### Part Four: Orchestrating the Conversation

The `chat_interface` function acts as the **brain of the assistant**. It ties all the individual pieces together by deciding how user input is handled, whether that input comes from typed text or transcribed speech. Once the input is ready, it sends the message—along with conversation history—to the language model to generate a context-aware response.

After receiving the AI’s reply, the function not only displays it but also passes it to the text-to-speech system so the assistant can respond out loud. In this way, `chat_interface` coordinates listening, thinking, and speaking, turning separate capabilities into a smooth, human-like conversational experience.


```python
def chat_interface(text_input, chat_history, voice_choice, audio_file=None):
    # Get user input from text or audio
    user_input = text_input
    if audio_file:
        user_input = transcribe_audio(audio_file)
        print(f"Transcribed audio: {user_input}")

    if not user_input or not user_input.strip():
        return chat_history, ""
```


### Understanding the Parameters

The function is designed to handle both text-based and voice-based interactions, so it accepts several inputs that cover all possible user actions. The `text_input` parameter contains whatever the user typed into the chat box, if any. The `chat_history` parameter stores the full conversation so far, allowing the assistant to respond with proper context instead of treating each message in isolation. The `voice_choice` parameter controls which synthesized voice the assistant will use when speaking its response. Finally, `audio_file` is optional and represents a recorded voice message from the user, which may need to be transcribed before processing.

When the function runs, its first responsibility is to figure out **what the user actually said**. If an audio file is provided, the function transcribes it into text; otherwise, it falls back to the typed input. This unified text representation ensures the rest of the logic works the same way regardless of how the user communicated. If the resulting input is empty or contains only whitespace, the function exits immediately without calling the language model, preventing unnecessary API calls and keeping the chatbot responsive and efficient.


```python
    if chat_history is None:
        chat_history = []

    messages = [{"role": "system", "content": "You are a helpful, natural AI assistant."}]

    for msg in chat_history:
        if isinstance(msg, dict):
            messages.append(msg)
        elif isinstance(msg, (list, tuple)) and len(msg) == 2:
            messages.append({"role": "user", "content": msg[0]})
            messages.append({"role": "assistant", "content": msg[1]})

    messages.append({"role": "user", "content": user_input})
```


### Building the Conversation Context

The function begins by ensuring there is a conversation history to work with. If this is the user’s first message, an empty history is initialized so the assistant has a clean starting point. A new `messages` list is then created, beginning with a **system message** that defines the assistant’s role, tone, and behavior. This system message acts as the “personality anchor” for the AI and is always included at the start of the conversation sent to OpenAI.

Next, the function loops through the existing chat history and converts each turn into the format expected by OpenAI. This step is necessary because different versions of Gradio may store conversation data differently—sometimes as dictionaries and sometimes as lists or tuples. The function handles both cases defensively, extracting the user and assistant messages and transforming them into dictionaries with explicit `"role"` and `"content"` keys. Finally, the user’s new message is appended to the end of the `messages` list, ensuring the model sees the full, correctly structured conversation context before generating its response.

This conversion layer is crucial because Gradio and OpenAI represent conversations differently. OpenAI’s API requires a strict message schema, while Gradio prioritizes UI flexibility. Bridging this gap ensures the assistant remains coherent, context-aware, and compatible across different frontend implementations.


```python
try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages
        )
        ai_reply = response.choices[0].message.content
    except Exception as e:
        ai_reply = f"Error: {e}"

    speak(ai_reply, voice_choice)

    chat_history.append({"role": "user", "content": user_input})
    chat_history.append({"role": "assistant", "content": ai_reply})

    return chat_history, ""
```


### Getting and Delivering the Response

Once the conversation context is fully assembled, all messages are sent to OpenAI’s **GPT-4o-mini** model to generate the assistant’s reply. The function then extracts the AI’s text response from the API output, isolating just the content that should be shown and spoken. If anything goes wrong during this process—such as a network issue or API error—the function gracefully falls back to creating a readable error message instead of crashing the application.

After a valid response is obtained, the assistant immediately calls the `speak()` function to convert the text into natural-sounding speech and play it aloud. Both the user’s message and the AI’s response are then appended to the chat history, preserving conversational continuity for future turns. Finally, the function returns the updated chat history along with an empty string, which Gradio interprets as a signal to clear the text input box and prepare the interface for the next message.

### Part Five: Building the User Interface

With the conversational logic in place, the final step is constructing the visual interface. **Gradio Blocks** provides a flexible layout system that allows you to precisely arrange UI components such as chat windows, text inputs, audio uploaders, and buttons. Unlike simpler Gradio interfaces, Blocks lets you control structure and flow explicitly, making it ideal for building a polished, chatbot-like experience that feels responsive and intuitive to users.


```python
with gr.Blocks() as demo:
    gr.Markdown("# 🎙️ Human-Like AI Voice Assistant")
    gr.Markdown("Type or speak your message, and the AI will respond with voice!")

    chatbot = gr.Chatbot()
```


### Creating the Foundation

The application begins inside a `with gr.Blocks()` context, which establishes the overall structure of your Gradio app and allows you to define all interface components in a clean, organized way. Inside this block, the first elements added are formatted text components using Markdown. The `#` symbol creates a large, prominent heading, which is useful for clearly presenting the title or purpose of your assistant to the user.

Next, `gr.Chatbot()` is used to create the main conversation display area. This component is responsible for visually rendering the back-and-forth dialogue between the user and the AI, showing messages in a familiar chat-style format. It automatically updates as new messages are added, giving the assistant the look and feel of a real-time conversational agent.


```python
with gr.Row():
        with gr.Column(scale=4):
            txt = gr.Textbox(
                placeholder="Type your message here...",
                label="Text Input",
                lines=2
            )
        with gr.Column(scale=1):
            voice = gr.Dropdown(
                ["alloy", "echo", "fable", "onyx", "nova", "shimmer"],
                value="alloy",
                label="Voice"
            )
```


### Organizing the Input Controls

The input section is organized using `gr.Row()`, which places components horizontally on the same line. Inside this row, two columns are defined to divide the available space in a clear and balanced way. The first column uses a larger scale value, taking roughly four-fifths of the width, while the second column uses a smaller scale, occupying the remaining one-fifth. This proportional layout is controlled by the `scale` parameter, which determines how much horizontal space each column receives relative to the others.

The wider column contains the main text input box, set to two lines tall so users can comfortably type short messages or questions. The narrower column holds a dropdown menu that allows the user to select the AI’s voice. Placing the voice selector alongside the text input keeps all interaction controls in one compact area, making the interface intuitive while preserving a clean, chat-like experience.


```python
    audio_input = gr.Audio(label="Or record/upload audio", type="filepath")

    send_btn = gr.Button("Send", variant="primary")
```


### Adding More Controls

Additional input controls enhance the interactivity of the chat interface. The `gr.Audio()` component allows users to either record their voice directly or upload an existing audio file. By setting `type="filepath"`, Gradio passes the path to the saved audio file to your backend function, making it easy to process with the AI. This enables seamless voice-based interactions without requiring manual file handling.

A `gr.Button()` component acts as the send button for the chat. Using `variant="primary"` makes the button visually prominent, signaling to users that it is the main action to submit their message. Together, the audio input and send button provide multiple ways for users to communicate with the AI, whether by speaking or typing, improving accessibility and user experience.


```python
send_btn.click(
        chat_interface,
        inputs=[txt, chatbot, voice, audio_input],
        outputs=[chatbot, txt]
    )

    txt.submit(
        chat_interface,
        inputs=[txt, chatbot, voice, audio_input],
        outputs=[chatbot, txt]
    )

demo.launch()
```


### Making It Interactive

Interactivity in Gradio is achieved by connecting UI components to backend functions. The `send_btn.click()` method specifies that **when someone clicks the Send button, the `chat_interface` function should be called**. The `inputs` list tells Gradio which components provide data to the function—such as the text input, chat history, selected voice, or audio file—while the `outputs` list defines which components get updated with the function’s return values, typically the chatbot display and the text input box.

Similarly, `txt.submit()` allows users to **trigger the same function by pressing Enter** in the text box, enabling a more natural chat experience. Finally, `demo.launch()` starts a local web server and opens the interface in a browser. This setup ensures that whenever a user types a message or records audio, Gradio:

1. Collects values from the input components.
2. Calls the `chat_interface` function with these values.
3. Updates the output components with the AI’s response.
4. Refreshes the chat display in real-time.

This clear data flow creates a smooth, responsive, and human-like chat interaction.
